In [6]:
import tensorflow as tf
from google_images_download import google_images_download
from glob import glob

In [3]:
response = google_images_download.googleimagesdownload()

In [5]:
response.download({"keywords":"Trump"})


Item no.: 1 --> Item name = Trump
Evaluating...
Starting Download...
Completed Image ====> 1.1200px-Donald_Trump_official_portrait.jpg
Completed Image ====> 2.Trump-1.jpg
Completed Image ====> 3.shutterstock_editorial_10434333bm.jpg
Invalid or missing image format. Skipping...
Invalid or missing image format. Skipping...
Completed Image ====> 4.191126115106-01-trump-1126-exlarge-169.jpg
Completed Image ====> 5.p07xty0h.jpg
Completed Image ====> 6.7c97e4730a334776bbb418bed4d563f2_18.jpg
Completed Image ====> 7.1ab3a9e580414748a9772bdab2d1dfb5_18.jpg
Completed Image ====> 8.106146633-1569349875620rtx74f71.jpg
Completed Image ====> 9.merlin_166097061_245e2dea-f9b9-4442-a8e2-af0c7cf7744c-articleLarge.jpg
Invalid or missing image format. Skipping...
Invalid or missing image format. Skipping...
Completed Image ====> 10.1165378207.jpg
Completed Image ====> 11.190924152430-02-trump-unga-0924-exlarge-169.jpg
Completed Image ====> 12.donald-trump-august-2019-e1566168925130.jpg
Completed Image =

({'Trump': ['/Users/jsimluken/Documents/pyTraining/tensorflow/tfv2/tf_data/downloads/Trump/1.1200px-Donald_Trump_official_portrait.jpg',
   '/Users/jsimluken/Documents/pyTraining/tensorflow/tfv2/tf_data/downloads/Trump/2.Trump-1.jpg',
   '/Users/jsimluken/Documents/pyTraining/tensorflow/tfv2/tf_data/downloads/Trump/3.shutterstock_editorial_10434333bm.jpg',
   '/Users/jsimluken/Documents/pyTraining/tensorflow/tfv2/tf_data/downloads/Trump/4.191126115106-01-trump-1126-exlarge-169.jpg',
   '/Users/jsimluken/Documents/pyTraining/tensorflow/tfv2/tf_data/downloads/Trump/5.p07xty0h.jpg',
   '/Users/jsimluken/Documents/pyTraining/tensorflow/tfv2/tf_data/downloads/Trump/6.7c97e4730a334776bbb418bed4d563f2_18.jpg',
   '/Users/jsimluken/Documents/pyTraining/tensorflow/tfv2/tf_data/downloads/Trump/7.1ab3a9e580414748a9772bdab2d1dfb5_18.jpg',
   '/Users/jsimluken/Documents/pyTraining/tensorflow/tfv2/tf_data/downloads/Trump/8.106146633-1569349875620rtx74f71.jpg',
   '/Users/jsimluken/Documents/pyTraini

In [7]:
files = glob("downloads/Trump/*")

In [12]:
raw = tf.io.read_file(files[0])
files[0]

'downloads/Trump/58.hv5soju7_400x400.jpg'

In [44]:
def make_dataset(files):
    dataset = tf.data.Dataset.from_tensor_slices(files)
    
    def _load_file(file):
        name = tf.strings.split(file,"/")[-1]
        id_ = tf.strings.split(name,".")[0]
        id_ = tf.strings.to_number(id_)
        raw = tf.io.read_file(file)
        img_tensor = tf.image.decode_image(raw)
        return img_tensor,id_
    dataset = dataset.map(_load_file)
    return dataset

In [47]:
dataset = make_dataset(files)
dataset

<MapDataset shapes: (<unknown>, ()), types: (tf.uint8, tf.float32)>

In [49]:
for img,id_ in dataset.take(10):
    print(id_)

tf.Tensor(58.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(63.0, shape=(), dtype=float32)
tf.Tensor(46.0, shape=(), dtype=float32)
tf.Tensor(60.0, shape=(), dtype=float32)
tf.Tensor(70.0, shape=(), dtype=float32)
tf.Tensor(64.0, shape=(), dtype=float32)
tf.Tensor(40.0, shape=(), dtype=float32)
tf.Tensor(31.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)


In [51]:
tfrec.write(dataset)

AttributeError: 'tuple' object has no attribute 'is_compatible_with'

In [52]:
dataset

<MapDataset shapes: (<unknown>, ()), types: (tf.uint8, tf.float32)>

In [101]:
def to_tfrecord(files):
    dataset = tf.data.Dataset.from_tensor_slices(files)
    def _load(file):
        name = tf.strings.split(file,"/")[-1]
        id_ = tf.strings.split(name,".")[0]
        id_ = tf.strings.to_number(id_)
        raw = tf.io.read_file(file)
        return raw,id_

    def _serialize(raw,id_):
        feature = {
            "image":tf.train.Feature(bytes_list=tf.train.BytesList(value=[raw.numpy()])),
            "id":tf.train.Feature(float_list=tf.train.FloatList(value=[id_]))
        }
        example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
        return example_proto.SerializeToString()
    def tf_serialize_example(raw,id_):
        tf_string = tf.py_function(
            _serialize, 
            (raw,id_),  # 上記の関数にこれらの引数を渡す
            tf.string)      # 戻り値の型は tf.string
        return tf.reshape(tf_string, ())
        
    dataset = dataset.map(_load).map(tf_serialize_example)
    
    return dataset
    

In [102]:
serialized_dataset = to_tfrecord(files)
serialized_dataset

<MapDataset shapes: (), types: tf.string>

In [108]:
for d in serialized_dataset.take(1):
    example = tf.train.Example.FromString(d)
    print(example)

/Users/jsimluken/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  


In [110]:
writer =  tf.data.experimental.TFRecordWriter('images.tfrec')
writer.write(serialized_dataset)

In [111]:
!du -h images.tfrec

 14M	images.tfrec


In [167]:
@tf.function
def load_tfrecord(path="images.tfrec"):
    dataset = tf.data.TFRecordDataset(path)
    def _parse(example_proto):
        feature_description = {
            "image":tf.io.FixedLenFeature([], tf.string,),
            "id":tf.io.FixedLenFeature([1], tf.float32, default_value=0.0),
        }
        example =  tf.io.parse_single_example(example_proto, feature_description)
        return tf.image.decode_image(example["image"]),example["id"]
    dataset = dataset.map(_parse)
    return dataset

In [168]:
dataset = load_tfrecord()
dataset

<_VariantDataset shapes: (<unknown>, (1,)), types: (tf.uint8, tf.float32)>

In [172]:
for img,id_ in dataset.take(10):
    print(img.shape,id_)

(400, 400, 3) tf.Tensor([58.], shape=(1,), dtype=float32)
(1520, 1200, 3) tf.Tensor([1.], shape=(1,), dtype=float32)
(1088, 727, 3) tf.Tensor([63.], shape=(1,), dtype=float32)
(800, 1200, 3) tf.Tensor([46.], shape=(1,), dtype=float32)
(381, 678, 3) tf.Tensor([60.], shape=(1,), dtype=float32)
(556, 989, 3) tf.Tensor([70.], shape=(1,), dtype=float32)
(383, 681, 3) tf.Tensor([64.], shape=(1,), dtype=float32)
(599, 800, 3) tf.Tensor([40.], shape=(1,), dtype=float32)
(683, 1024, 3) tf.Tensor([31.], shape=(1,), dtype=float32)
(698, 1240, 3) tf.Tensor([2.], shape=(1,), dtype=float32)
